# How do you make a difference from within the Berlin tech scene?
## A.K.A. How do I land the best programming job in Berlin?

- Life is too short to be spent in front of a computer if that time is not bringing significant value to other people's lives. For me personally, 
- I've meandered through various professional environments which, although at times have been fulfilling, have ultimately had very little positive impact on the world. 
- On the hunt for my next tech job, I want to take a different approach this time: I want to expand my search drastically to find out just which company will suit me best.
- I want to understand the job market in Berlin (the city in which I reside) so that I can, with some degree of confidence, say I'm doing the best I can from within this city. 

## A few guiding questions:
- What programming, data or just generally tech roles are out there?
- What different sectors exist out there (e.g. mobility, e-commerce, rental market etc.)?
- What are the average salaries for different roles?
- What companies currently have a positive impact?
- What companies provide the best salaries?
- What companies provide the best working environments?
- What does it mean to be a senior developer? How does one get there?
- What skill set is most in demand and in which fields? 

## Data sources:
- Scraping various job-posting websites (e.g. Glassdoor, LinkedIn)
- https://berlinstartupjobs.com/
- https://www.gehalt.de/
- https://berlinvalley.com/
- https://www.tagesspiegel.de/themen/gruenderzeit/

## Data collection (Glassdoor)
### Cleaning

In [3]:
import pandas as pd
import re

#Read in the latest job scrape(s)
df = pd.read_json('output/glassdoor-jobs_2021-06-13_12:52:22.json')
df2 = pd.read_json('output/glassdoor-jobs_2021-06-14_15:29:42.json')
df = pd.concat([df, df2])


#Remove the column name prefixes (artefacts from scraping process)
def strip_pref(x):
    return x[len('data_jobView_header_'):] if 'data_jobView_header_' in x else x[len('data_jobView_'):]
df.columns = [strip_pref(p) for p in df.columns]

#Let's for now just focus on employer, job title and job description. The other things might come in handy later
column_names = [
    'employer_name',
    'normalizedJobTitle',
    'job_description',
    #'ageInDays',
    #'employer_size',
    #'expired',
    #'locationName',
    #'job_listingId'
]
df = df[column_names]

#And normalise the column names once more into snake_case
pattern = re.compile(r'(?<!^)(?=[A-Z])')
df.columns = [pattern.sub('_', name).lower() for name in df.columns]

##Drop any rows that do not have one of our three fields
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 834 entries, 0 to 599
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   employer_name         834 non-null    object
 1   normalized_job_title  834 non-null    object
 2   job_description       834 non-null    object
dtypes: object(3)
memory usage: 26.1+ KB


**TODO:** Since we've only performed one search at the moment, I'm going to assume that there are no duplicate job postings. This will have to be fixed once the scraper performs several different search queries.

## Exploration
### How many companies are hiring and what types of positions are there?
Now that we have a decent list of open job positions, let's take a look at the companies that are hiring and the positions that are open:

In [20]:
[print(n) for n in df['employer_name'].value_counts().index.values];

Delivery Hero GmbH
CapGemini SE
Zalando
Bayer Corporation
M2. technology & project consulting GmbH
Wayfair 
Publicis Sapient
HelloFresh
Amazon.com, Inc.
Deloitte 
Campusjäger GmbH
ixto GmbH 
FlixBus GmbH
GetYourGuide AG 
REPLY S.p.A
Enpal GmbH
PKW1.net GmbH
idealo internet GmbH
Adecco Group
BearingPoint Holding B.V
Smart Steel Technologies GmbH
Trade Republic Bank GmbH
UP42 GmbH
vantago
PwC
future demand
MTR Travel Technologies UG (haftungsbeschränkt)
OptioPay
Forto GmbH
Accenture Ltd
CORE 
Wall Ag
home24 AG
ZAGENO, Inc. 
Adesso SE
Una Health
Mobility Trader GmbH
The Boston Consulting Group Inc.
Wooga GmbH
msg global solutions ag
BuildingMinds Inc.
Cocomore AG
HERE Global B.V.
orderbird
PriceHubble AG
enersis suisse ag
Moonfare GmbH 
Sparkassen-Finanzportal GmbH
RepRisk AG
act. 3
Axel Springer SE
Global Savings Group
JCFINCH Executive & Talent Search | Christian Finke
Bain & Company, Inc.
ThoughtWorks, Inc.
Exxeta AG
Pair Finance GmbH
TIER Mobility GmbH
Deutsche Bahn Aktiengesellschaft

In [17]:
[print(n) for n in df['normalized_job_title'].value_counts().index.values];

Data Engineer
Senior Data Engineer
Datenwissenschaftler (M/W/D)
Senior Consulting Manager (M/W/D)
Berater (M/W/D)
Data Scientist
Senior Data Scientist
Data Analyst
Dateningenieur (M/W/D)
Senior Data Analyst
Senior Product Manager
Datenanalyst (M/W/D)
Machine Learning Engineer
Product Manager
Senior Berater (M/W/D)
Senior Machine Learning Engineer
Data Scientist Manager
Senior Datenwissenschaftler (M/W/D)
Cloud Engineer (M/W/D)
Senior Back End Engineer
Software Engineer
Senior Manager Data Scientist
Full Stack Engineer
Business Analyst (M/W/D)
Machine Learning Engineer (M/W/D)
Operations Analyst
Senior Manager Systems Engineer
Lead Data Engineer
Staff Data Engineer
Head Product Manager
Senior Materials Scientist
Project Manager
Product Owner
Senior Devops Engineer
Manager
Cloud Engineer
Associate Intern
Engineer Manager
Principal Software Engineer
Junior Data Analyst
Senior Software Engineer
Devops Engineer (M/W/D)
Visual Manager
Data Engineer Intern
Junior Dateningenieur (M/W/D)
Senior

In [25]:
gp = df.groupby('employer_name')['normalized_job_title'].apply(list).to_frame()
with open('misc/test3.html', 'w') as f:
    f.write(gp.to_html(escape=False))

So, it looks like the company names could be useful just as they are. The position titles will need some normalising (translating to English, removing 'M/W/D', etc.) 

For now, let's see if we can move onto a more interesting question:

### How many different sectors of the economy are represented by these companies?
A key insight into landing a job that suits me best will be to get a better idea of what sectors of the economy are currently looking for more developers, and then use that information to a) find sectors I might not have thought about and b) know if I'm trying to get into a sector that's going to be more competetive. Since we have a tidy list of company names, this should be possible.

## How many senior positions are there?

In [2]:
contains_sr = df['job_description'].str.contains(r'senior')
df_sr = df[contains_sr]
with open('misc/test.html', 'w') as f:
    f.write(df.to_html(escape=False))

In [4]:
dfli = pd.read_csv('linkedin-jobs.csv')
dfli.head()

with open('misc/test2.html', 'w') as f:
    f.write(dfli.to_html(escape=False))